In [1]:
import os
from os import listdir
from os.path import isdir, join
import sys

# Setting Directories

In [2]:
data_dir = '/content/drive/MyDrive/MagNetDrive'  # If using Colab with Google Drive, set this to the location on Drive containing all the data for this project ie '/MyDrive/MagNetDatastore'

# Setting Environment

In [3]:
# Check if the code is running on Google Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    # Mount Google Drive
    drive.mount('/content/drive')
    # Set the dataset directory path on Google Drive
    data_dir = os.path.join('/content/drive/', data_dir)  # Changes current working directory to Google Drive hosted folder, containing all data and functional scripts
    os.chdir(data_dir)
    platform = "colab"
    print("Running on Google Colab. Google Drive mounted.")
else:
    platform = "local"
    print("Running locally.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running on Google Colab. Google Drive mounted.


# Loading Training Materials

In [4]:
# get all folders name in current directory
preprocessed_training_dataset_path = os.path.join(data_dir, "preprocessed_training_dataset")

# Initialize the training_materials list
training_materials = []

# Check if the "preprocessed_training_dataset" folder exists
if os.path.exists(preprocessed_training_dataset_path) and os.path.isdir(preprocessed_training_dataset_path):
    # Get the list of materials in the "preprocessed_training_dataset" folder
    subfolders = [item for item in os.listdir(preprocessed_training_dataset_path)
                  if os.path.isdir(os.path.join(preprocessed_training_dataset_path, item))]

    # Check if there are any subfolders
    if subfolders:
        # Save the names of the subfolders to the training_materials list
        training_materials = subfolders
    else:
        print("No training dataset found in dataset directory, preprocessed (normalised, down sampled and split) training materials should be saved into subfolders within a folder called 'preprocessed_training_dataset', where each subfolder is the name of a training material.")
else:
    print("No training dataset found in dataset directory, preprocessed (normalised, down sampled and split) training materials should be saved into subfolders within a folder called 'preprocessed_training_dataset', where each subfolder is the name of a training material.")

# Print the list of training materials if they exist
if training_materials:
    print("Identified training data for materials:", training_materials)

Identified training data for materials: ['3C90', 'N87', '3E6', 'N49', '3F4', '77', '78', 'N27', 'N30', '3C94']


In [5]:
import torch

gpu_num = 0
cuda_ready = False # Track if CUDA hardware acceleration is engaged

if torch.cuda.is_available():
    cuda_ready = True
    print('CUDA available!')
    gpu_num = torch.cuda.device_count()
    if (gpu_num < 1):
        print('GPU unavailable')
    else:
        print('GPU num: ', gpu_num)  # Print number of GPUs
        for gpu in range(gpu_num):
            print('GPU type: ', torch.cuda.get_device_name(gpu))  # Print model of GPU
            print('GPU memory: {:.2f} Gbyte'.format(
                torch.cuda.get_device_properties(gpu).total_memory /
                1e9))  # Print total GPU memory
else:
    cuda_ready = False
    print('CUDA unavailable')

#print(platform)
#print(os.getcwd())
#print(cuda_ready)
#print(os.path.abspath(''))

# Check if CUDA is available and if so, set the device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print("System using ", device)

CUDA available!
GPU num:  1
GPU type:  Tesla T4
GPU memory: 15.84 Gbyte
System using  cuda


# Model Training
First set base material for transfer learning

In [6]:
base_mat = "3C90" # This is the base material used to facilitate the first stage of transfer learning as described in the paper, material with largest training dataset selected by default, set as '' to train each material from scratch
model_saved_name = "model_weights.ckpt"

# Setting training period approproate for compute available
if cuda_ready:
    epochs = 500
    valid_batch_size = 3000
else:
    epochs = 20
    valid_batch_size = 1000

from train_model import train_model

if base_mat == '':
    print('Transfer learning function disabled, each material will train from zero initial weightings.')
    for material in training_materials:
        train_model(preprocessed_training_dataset_path, material, base_mat, model_saved_name, platform, device, epochs, valid_batch_size)
elif base_mat in training_materials:
    base_model_path = os.path.join(preprocessed_training_dataset_path, base_mat, model_saved_name)
    if os.path.exists(base_model_path):
        print('Base model used for transfer learning found at:', base_model_path)
        for material in training_materials:
            if material != base_mat:
                train_model(preprocessed_training_dataset_path, material, base_mat, model_saved_name, platform, device, epochs, valid_batch_size, verbose=False, load_pretrained=True)
    else:
        print(f"No base model of material {base_mat} found to facilitate transfer learning, program will need to train {base_mat} from scratch before fine-tuning for other materials.")
        train_model(preprocessed_training_dataset_path, base_mat, base_mat, model_saved_name, platform, device, epochs, valid_batch_size, verbose=True)
        for material in training_materials:
            if material != base_mat:
                train_model(preprocessed_training_dataset_path, material, base_mat, model_saved_name, platform, device, epochs, valid_batch_size, verbose=False, load_pretrained=True)
else:
    raise RuntimeError(f"No training data or pretrained model found for material {base_mat}, check that data directory and base material preference is correctly set in beginning of this cell.")

No base model of material 3C90 found to facilitate transfer learning, program will need to train 3C90 from scratch before fine-tuning for other materials.
LSTMSeq2One(
  (lstm): LSTM(1, 30, num_layers=3, batch_first=True)
  (fc1): Linear(in_features=32, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=196, bias=True)
  (fc3): Linear(in_features=196, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=96, bias=True)
  (fc5): Linear(in_features=96, out_features=32, bias=True)
  (fc6): Linear(in_features=32, out_features=32, bias=True)
  (fc7): Linear(in_features=32, out_features=16, bias=True)
  (fc8): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (leaky_relu): LeakyReLU(negative_slope=0.01)
  (elu): ELU(alpha=1.0)
  (sigmoid): Sigmoid()
)
Total number of parameters:  90653
  3C90 Model saved , Validation Loss: 7.488e-01, lr: 2.000e-05
  3C90 Model saved , Validation Loss: 7.428e-01, lr: 1.998e-05
  3C90 Model saved 